<a href="https://colab.research.google.com/github/Lily10938001/API-practice/blob/main/API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 串接API到自己的網域

In [ ]:
!pip install requests

In [ ]:
import requests
from datetime import datetime

def get_symbols_info(symbols):
    headers = {'x-api-key': "ifv7N4Itrtz6uCTkwN8G6fFxvoVV4643QsrNzg3d"}
    url = "https://yfapi.net/v8/finance/spark"
    querystring = {"symbols": symbols, "range": "1mo", "interval": "1d"}
    response = requests.request("GET", url, headers=headers, params=querystring)
    data = response.json()
    results = {}

    for symbol in symbols.split(","):
        # Convert timestamp to str datetime
        timestamps = data[symbol]["timestamp"]
        for index, timestamp in enumerate(timestamps):
            timestamps[index] = datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d')

        close_piece = data[symbol]["close"]
  
        symbol_info_mapping = dict(zip(timestamps, close_piece))  #將兩個list合成一個dict

        results[symbol] = symbol_info_mapping
    return results

if __name__ == '__main__':
    print(get_symbols_info("AAPL,MSFT")) 

## 串接MYSQL資料到自己的網域,並直接透過修改網域名稱對資料庫進行動作(RESTful API的特色) : 
### 工具一－MAMP: 
- https://www.mamp.info/en/windows/
- 可以用localhost連到網頁版的PhpMyadmin
- 按開關”Start Servers”　→　按中間”Open Webstart Page”　→　連到PhpMyAdmin(port號 = 3360)

### 工具二－POSTMAN:
- https://www.postman.com/downloads/
- 類似瀏覽器,貼上url就可以查詢所串的MYSQL資料庫api,但更能get、post、delete、update資料進去

In [ ]:
pip install flask # WEB框架
pip install flask-restful # 可把URL名稱寫得簡略，並將動作藏在HTTP的method裡面
pip install pymysql

In [ ]:
# app.py
# 試圖直接透過修改自己的網域名稱(在postman上操作)
# 來對串接的後台資料庫進行修改、查詢等功能

import flask
from flask_restful import Api  
from resource.user import Users, User  # resource(資料夾).user(檔名)


app = flask.Flask(__name__)
app.config["DEBUG"] = True
api = Api(app)

# 用restful api的形式為url命名
api.add_resource(Users, "/users")  # 對全體資料做動作
api.add_resource(User, "/user/<int:id>")  # 對指定的id那筆資料做動作


# py3預設的名字為main,此行是確保事py3是由name執行
if __name__ == '__main__':
    # flask server預定的localhost ip ,80誒是域定的port號
    app.run(host='127.0.0.1', port=10009)

# 在postman查詢http://127.0.0.1:10009/users會出現users的表格內容
# 在postman查詢http://127.0.0.1:10009/user/2會針對id=2的項目做相關動作

In [ ]:
# user.py
# 針對mysql(db名test、table名user)的資料做新增、刪除、查詢等動作的設計
# 先讓自己連上資料庫(phpmyadmin)

from flask import jsonify
from flask_restful import Resource, reqparse
import pymysql

class Users(Resource):
  def db_init(self):
      db = pymysql.connect(
          host='127.0.0.1',
          user='root',
          password='root',
          port=3306,
          db='test'
      )
      cursor = db.cursor(pymysql.cursors.DictCursor)
      return db, cursor


  def get(self):
      db, cursor = self.db_init()
      parser = reqparse.RequestParser()
      parser.add_argument('name')
      args = parser.parse_args()
      name = args.get("name")
      if name is not None:
          sql = f"SELECT * FROM test.user WHERE name = '{name}';"
      else:
          sql = 'SELECT * FROM test.user;'

      cursor.execute(sql)
      users = cursor.fetchall()
      db.close()
      return jsonify(users)
      # 在postman用GET查詢http://127.0.0.1:10009/users會出現users的表格內容

  def post(self):
      db, cursor = self.db_init()
      parser = reqparse.RequestParser()
      parser.add_argument('name', required=True)
      parser.add_argument('gender', required=True)
      parser.add_argument('birth')
      parser.add_argument('note')
      args = parser.parse_args()
      user = {
          'name': args['name'],
          'gender': args['gender'],
          'birth': args.get('birth') or '1900-01-01',
          'note': args.get('note')
      }
      sql = """

      INSERT INTO `test`.`user` (`name`,`gender`,`birth`,`note`)
      VALUES ('{}','{}','{}','{}');

      """.format(user['name'], user['gender'], user['birth'], user['note'])
      result = cursor.execute(sql)
      db.commit()
      db.close()
      response = {"message": "success"}
      return jsonify(response)
      # 在postman用POST查詢http://127.0.0.1:10009/users並在"body"-"form data"輸入post的資料就可以新增資料

class User(Resource):
  def db_init(self):
      db = pymysql.connect(
          host='127.0.0.1',
          user='root',
          password='root',
          port=3306,
          db='test'
      )

      cursor = db.cursor(pymysql.cursors.DictCursor)
      return db, cursor

  # Get single by id
  def get(self, id):
      db, cursor = self.db_init()
      print(db, cursor)
      sql = f"SELECT * FROM test.user WHERE id = '{id}';"
      cursor.execute(sql)
      users = cursor.fetchall()
      db.close()
      return jsonify(users)
      # 在postman用GET查詢http://127.0.0.1:10009/user/1就可以查詢到id=1的資料

  def patch(self, id):
      db, cursor = self.db_init()
      parser = reqparse.RequestParser()
      parser.add_argument('name')
      parser.add_argument('gender')
      parser.add_argument('birth')
      parser.add_argument('note')
      args = parser.parse_args()
      user = {
          'name': args.get('name'),
          'gender': args.get('gender'),
          'birth': args.get('birth'),
          'note': args.get('note')
      }

      query = []
      for key, value in user.items():
          if value is not None:
              query.append(f"{key} = '{value}'")
      query = ",".join(query)

      sql = """
          UPDATE test.user
          SET {}
          WHERE id = {};
      """.format(query, id)

      print(sql)
      result = cursor.execute(sql)
      db.commit()
      db.close()

      response = {"message": "success"}
      
      return jsonify(response)
      # 在postman用PATCH查詢http://127.0.0.1:10009/user/1並在"body"-"form data"輸入修改後的資料就可以修改id=1的資料
  
  def delete(self, id):
      db, cursor = self.db_init()
      sql = f'DELETE FROM `test`.`user` WHERE id = {id};'
      result = cursor.execute(sql)
      db.commit()
      db.close()

      response = {"message": "success"}
      return jsonify(response)
      # 在postman用DELETE查詢http://127.0.0.1:10009/user/1就可以刪除id=1的資料

In [ ]:
pip install flask-apispec
pip install flask-jwt-extended

In [ ]:
# decorator"@"

def introduction(func):   # func是等等要放入的get_number()
  def wrapper():  
    print ("這是一個會回傳int的函式")
    func()      # 回傳get_number()
  return (wrapper)   # 執行wrapper() #不用括號ㄟ

@introduction # 定義每次使用get_number()的時候都要做introduction()的所有動作
def get_number():
  print("1")

if __name__ == "__main__":
  get_number() 


這是一個會回傳int的函式
1


In [ ]:
def introduction(func):
   def wrap():
       print("heyhey")
       func()
   return wrap
 
def introduction_with_param(param1):
   def introduction(func):
       def wrap():
           print(param1)
           func()
          
       return wrap
   return introduction
  
param1 = "param1"      
 
@introduction
@introduction_with_param(param1)
def get_number():
   print("hey")

if __name__ == "__main__":
  get_number()

heyhey
param1
hey
